In [2]:
from pathlib import Path
import pandas as pd
from tqdm import tqdm
from datetime import datetime, timedelta
import numpy as np

In [3]:
path = Path('.')
dataframes = {}
columns =  ['dates','subframe_n','subframe_ind','rnti','direction','mcs']
for p in list(path.glob('new_A')):
    for file in tqdm(list(p.glob('*.csv*'))):
        df = pd.read_csv(file, names = columns)
        with pd.option_context('mode.use_inf_as_null', True):
            df = df.dropna()#fillna(0)#dropna(subset=['subframe_ind', 'rnti', 'direction', 'mcs'], how='all')
        df[['subframe_ind', 'rnti', 'direction', 'mcs']] = df[['subframe_ind', 'rnti', 'direction', 'mcs']].astype(np.int)
        df.dates = pd.to_datetime(df.dates, format='%Y%m%d%H%M%S')
        dataframes[file.parts[-1]]=df#.sort_values('rnti').reset_index(drop = True)

df = pd.concat([dataframes[key] for key in dataframes.keys()])

100%|██████████| 42/42 [01:10<00:00,  1.93it/s]


In [4]:
df

,dates,subframe_n,subframe_ind,rnti,direction,mcs
0,2017-03-23 00:00:00,28,4,46997,1,20
1,2017-03-23 00:00:01,53,5,32000,1,20
2,2017-03-23 00:00:01,85,3,45928,1,0
3,2017-03-23 00:00:01,91,9,5,1,0
4,2017-03-23 00:00:02,27,8,22109,1,4
5,2017-03-23 00:00:02,67,0,7999,1,14
6,2017-03-23 00:00:02,75,2,26023,1,10
7,2017-03-23 00:00:03,93,5,49870,1,11
8,2017-03-23 00:00:05,18,8,57414,1,7
9,2017-03-23 00:00:06,23,8,16183,1,14


In [ ]:
# rnti is renewed only when 10 seconds pass
delta_rnti_renewal = timedelta(seconds = 10)

# list where single dataframe for each user is saved
users = []
# list where last time of wach user is saved
last_time = []

# structure where on first row the index is set to 1 when rnti has been found 
# and the second row save the idnex of users and last time lists
current_users = np.zeros((65536,2), dtype = np.int)

# tqdm progress bar
with tqdm(total=len(df)) as pbar:
    for index, row in df.iterrows():
        # if already found user
        if current_users[row.rnti,0]:
            # if the current row is generated by the same rnti because the date is nearer than 10s
            if row.dates < last_time[current_users[row.rnti,1]] + delta_rnti_renewal:
                # already existing user so append the current row 
                users[current_users[row.rnti,1]].append(row)
                # save the last timestep
                last_time[current_users[row.rnti,1]] = row.dates

            else:
                # user found but the rnti has been renewed so append new data
                users.append([row])
                last_time.append(row.dates)

                current_users[row.rnti,1] = len(users) - 1
        else:
            # user not found so flag the rnti index and append the new data
            users.append([row])
            last_time.append(row.dates)

            current_users[row.rnti,0] = 1
            current_users[row.rnti,1] = len(users) - 1
        pbar.update(1)

for index, user in enumerate(users):
    users[index] = pd.DataFrame(user)
print('Found {} users'.format(len(users)))

 52%|█████▏    | 5535765/10628508 [12:03<09:44, 8711.08it/s]  